In [55]:
from pathlib import Path
import pandas as pd
import re
import xlsxwriter

In [56]:
inputfolder = Path(r"C:\Users\jasonjia\Dropbox\Projects\channels_in_macro\data\paragraphs_channel")
inputfile = Path("paragraphs_channel_truncated.xlsx")
outputfolder = inputfolder
outputfile = Path("paragraphs_channel_truncated_bold.xlsx")

In [57]:
keywordlist = ['channel']

## Functions

In [58]:
def split_paragraph_by_keywordlist(keywordlist, paragraph, verbose = False):
    # Split paragraph by any keyword in a keyword list, where lower/uppercases are ignored, so it allows
    # any form of the keyword to appear in the paragraph, without having to convert
    # the paragraph to lowercase (and thus losing all the uppercases when saving it 
    # to excel later on).
    # The output will be a bunch of strings, that goes something like non-keyword chunk -> keyword -> non-keyword chunk.
    
    keywordlist_pattern = '|'.join(keywordlist)
    if verbose:
        print(keywordlist_pattern)
        
    paragraph_chunks = []
    start_index_chunk = 0
    
    matches = re.finditer(keywordlist_pattern, paragraph, flags=re.IGNORECASE)
    if verbose: 
        print(matches)

    number_of_keywords_in_paragraph = len(re.findall(keywordlist_pattern, paragraph, flags=re.IGNORECASE))
    print("number_of_keywords_in_paragraph: ", str(number_of_keywords_in_paragraph))

    for index, match in enumerate(matches):
        if verbose: 
            print(match.string)
            print(match.group())
            print(match.start())
            print(match.end())

        keyword = match.group()
        start_index_keyword = match.start()
        end_index_keyword = match.end()

        paragraph_chunk = paragraph[start_index_chunk:start_index_keyword]
        start_index_chunk = end_index_keyword
        paragraph_chunks.append(paragraph_chunk)
        paragraph_chunks.append(keyword)
        
        if verbose: 
            print("index: ", str(index))
            
        if index == (number_of_keywords_in_paragraph - 1):
            paragraph_chunk = paragraph[start_index_chunk:]
            paragraph_chunks.append(paragraph_chunk)

        if verbose: 
            print(paragraph_chunks)
                
    return paragraph_chunks

In [59]:
def get_formatted_paragraph_chunks(keywordlist, paragraph, verbose = False):
    # Split paragraph by keyword
    paragraph_chunks = split_paragraph_by_keywordlist(keywordlist, paragraph)
    if verbose == True:
        print("paragraph_chunks: ", paragraph_chunks) 

    # Keep only non-empty strings
    paragraph_chunks = list(filter(None, paragraph_chunks))  
    
    # Initialize final list of paragraph chunks with keyword bolded
    paragraph_chunks_formatted = []
    
    for chunk in paragraph_chunks:
        if verbose == True:
            print("chunk: ", chunk)
            
        if any(chunk.lower() == keyword for keyword in keywordlist):
            paragraph_chunks_formatted.append(bold_format_paragraph)
        else:
            paragraph_chunks_formatted.append(non_bold_format_paragraph)
        
        paragraph_chunks_formatted.append(chunk)
        if verbose == True:
            print("paragraph_chunks_formatted:", paragraph_chunks_formatted)
        

    if verbose == True:
        print("paragraph_chunks_formatted - final:", paragraph_chunks_formatted)

    return paragraph_chunks_formatted

## Code

In [66]:
# Import channels df
channels_df = pd.read_excel(Path(inputfolder / inputfile))

# Check that the last column is 'Paragraph', because it is needed for subsequent code to run correctly
if channels_df.columns.values[-1] != 'Paragraph':
    print("Warning: Last column of channels_df is not 'Paragraph'. Subsequent code won't work as expected.\n")

# Get last column of df (0-indexed)
paragraph_col_index_number = channels_df.shape[1] - 1
number_of_paragraphs = channels_df.shape[0] - 1

In [67]:
# Create workbook (output excel file)
workbook = xlsxwriter.Workbook(Path(outputfolder / outputfile))
worksheet = workbook.add_worksheet()

# Create bold format for column headers
bold_format_col_headers = workbook.add_format({'bold': True, "align":"center", "valign":"center"})

# Create bold format for keywords in paragraph
bold_format_paragraph = workbook.add_format({'bold': True})

# Create non-bold format for non-keywords in paragraph
non_bold_format_paragraph = workbook.add_format({'bold': False})

# Create wrap text format for column
wrap_text_format_col = workbook.add_format({'text_wrap': True})

# Set column widths - 'Paragraph' should be the last column
column_widths = [25, 30, 30, 10, 20, 140]

# Set row height
row_height = 15

# Set column width of all columns and turn wrap_text on
for index, width in enumerate(column_widths):
    worksheet.set_column(first_col = index, last_col = index, 
                         width = width, cell_format = wrap_text_format_col)
    
# Set row_heights of all rows
for i in range(number_of_paragraphs):
    worksheet.set_column(number_of_paragraphs + 1, row_height) # arguments: row, (row) height

In [68]:
# Write first row (column header)
worksheet.write_row(row = 0, col = 0, data = channels_df.columns.values, cell_format = bold_format_col_headers)

# Write rest of rows (paragraphs) - 'Paragraph' should be the last column
for index, row in channels_df.iterrows():
    paragraph = row['Paragraph']
    print(row['Title'])
    worksheet.write_row(row = index + 1, col = 0, data = row.drop('Paragraph'))
    paragraph_chunks_formatted = get_formatted_paragraph_chunks(keywordlist, paragraph)
    worksheet.write_rich_string(index + 1, paragraph_col_index_number, *paragraph_chunks_formatted)

workbook.close()

Sectoral Media Focus and Aggregate Fluctuations.
number_of_keywords_in_paragraph:  1
Sectoral Media Focus and Aggregate Fluctuations.
number_of_keywords_in_paragraph:  1
Sectoral Media Focus and Aggregate Fluctuations.
number_of_keywords_in_paragraph:  1
Sectoral Media Focus and Aggregate Fluctuations.
number_of_keywords_in_paragraph:  2
Product Innovation, Product Diversification, and Firm Growth: Evidence from Japan's Early Industrialization.
number_of_keywords_in_paragraph:  1
Product Innovation, Product Diversification, and Firm Growth: Evidence from Japan's Early Industrialization.
number_of_keywords_in_paragraph:  2
Venting Out: Exports during a Domestic Slump.
number_of_keywords_in_paragraph:  1
Venting Out: Exports during a Domestic Slump.
number_of_keywords_in_paragraph:  1
Venting Out: Exports during a Domestic Slump.
number_of_keywords_in_paragraph:  2
Venting Out: Exports during a Domestic Slump.
number_of_keywords_in_paragraph:  2
Labor Rationing.
number_of_keywords_in_par

Importing Political Polarization? The Electoral Consequences of Rising Trade Exposure.
number_of_keywords_in_paragraph:  2
Importing Political Polarization? The Electoral Consequences of Rising Trade Exposure.
number_of_keywords_in_paragraph:  1
Importing Political Polarization? The Electoral Consequences of Rising Trade Exposure.
number_of_keywords_in_paragraph:  1
Importing Political Polarization? The Electoral Consequences of Rising Trade Exposure.
number_of_keywords_in_paragraph:  2
Importing Political Polarization? The Electoral Consequences of Rising Trade Exposure.
number_of_keywords_in_paragraph:  1
Importing Political Polarization? The Electoral Consequences of Rising Trade Exposure.
number_of_keywords_in_paragraph:  1
Liquidity versus Wealth in Household Debt Obligations: Evidence from Housing Policy in the Great Recession.
number_of_keywords_in_paragraph:  1
Liquidity versus Wealth in Household Debt Obligations: Evidence from Housing Policy in the Great Recession.
number_of_

number_of_keywords_in_paragraph:  1
Monetary Policy According to HANK.
number_of_keywords_in_paragraph:  1
Monetary Policy According to HANK.
number_of_keywords_in_paragraph:  1
Monetary Policy According to HANK.
number_of_keywords_in_paragraph:  1
Monetary Policy According to HANK.
number_of_keywords_in_paragraph:  1
Monetary Policy According to HANK.
number_of_keywords_in_paragraph:  1
Export Destinations and Input Prices.
number_of_keywords_in_paragraph:  1
Export Destinations and Input Prices.
number_of_keywords_in_paragraph:  1
Export Destinations and Input Prices.
number_of_keywords_in_paragraph:  1
Export Destinations and Input Prices.
number_of_keywords_in_paragraph:  1
Export Destinations and Input Prices.
number_of_keywords_in_paragraph:  2
Export Destinations and Input Prices.
number_of_keywords_in_paragraph:  1
Export Destinations and Input Prices.
number_of_keywords_in_paragraph:  1
Export Destinations and Input Prices.
number_of_keywords_in_paragraph:  2
Export Destinatio

number_of_keywords_in_paragraph:  5
Breakthroughs, Deadlines, and Self-Reported Progress: Contracting for Multistage Projects†.
number_of_keywords_in_paragraph:  3
Breakthroughs, Deadlines, and Self-Reported Progress: Contracting for Multistage Projects†.
number_of_keywords_in_paragraph:  3
Breakthroughs, Deadlines, and Self-Reported Progress: Contracting for Multistage Projects†.
number_of_keywords_in_paragraph:  1
Breakthroughs, Deadlines, and Self-Reported Progress: Contracting for Multistage Projects†.
number_of_keywords_in_paragraph:  3
Understanding the Gains from Wage Flexibility: The Exchange Rate Connection†.
number_of_keywords_in_paragraph:  4
Understanding the Gains from Wage Flexibility: The Exchange Rate Connection†.
number_of_keywords_in_paragraph:  6
Understanding the Gains from Wage Flexibility: The Exchange Rate Connection†.
number_of_keywords_in_paragraph:  3
Understanding the Gains from Wage Flexibility: The Exchange Rate Connection†.
number_of_keywords_in_paragraph:

number_of_keywords_in_paragraph:  1
Diversity and Conflict.
number_of_keywords_in_paragraph:  2
Diversity and Conflict.
number_of_keywords_in_paragraph:  1
Diversity and Conflict.
number_of_keywords_in_paragraph:  2
Diversity and Conflict.
number_of_keywords_in_paragraph:  1
Heterogeneity and Persistence in Returns to Wealth.
number_of_keywords_in_paragraph:  1
Heterogeneity and Persistence in Returns to Wealth.
number_of_keywords_in_paragraph:  1
Heterogeneity and Persistence in Returns to Wealth.
number_of_keywords_in_paragraph:  1
Heterogeneity and Persistence in Returns to Wealth.
number_of_keywords_in_paragraph:  1
Informational Channels of Financial Contagion.
number_of_keywords_in_paragraph:  1
Informational Channels of Financial Contagion.
number_of_keywords_in_paragraph:  7
Informational Channels of Financial Contagion.
number_of_keywords_in_paragraph:  3
Informational Channels of Financial Contagion.
number_of_keywords_in_paragraph:  6
Informational Channels of Financial Cont

number_of_keywords_in_paragraph:  1
The Economic Origins of Conflict in Africa.
number_of_keywords_in_paragraph:  1
The Housing Boom and Bust: Model Meets Evidence.
number_of_keywords_in_paragraph:  1
The Housing Boom and Bust: Model Meets Evidence.
number_of_keywords_in_paragraph:  1
The Housing Boom and Bust: Model Meets Evidence.
number_of_keywords_in_paragraph:  1
The Housing Boom and Bust: Model Meets Evidence.
number_of_keywords_in_paragraph:  2
The Housing Boom and Bust: Model Meets Evidence.
number_of_keywords_in_paragraph:  1
The Housing Boom and Bust: Model Meets Evidence.
number_of_keywords_in_paragraph:  1
The Housing Boom and Bust: Model Meets Evidence.
number_of_keywords_in_paragraph:  1
Credit Market Disruptions and Liquidity Spillover Effects in the Supply Chain.
number_of_keywords_in_paragraph:  1
Credit Market Disruptions and Liquidity Spillover Effects in the Supply Chain.
number_of_keywords_in_paragraph:  3
Credit Market Disruptions and Liquidity Spillover Effects i

number_of_keywords_in_paragraph:  1
Estimating the Innovator's Dilemma: Structural Analysis of Creative Destruction in the Hard Disk Drive Industry, 1981-1998.
number_of_keywords_in_paragraph:  1
Estimating the Innovator's Dilemma: Structural Analysis of Creative Destruction in the Hard Disk Drive Industry, 1981-1998.
number_of_keywords_in_paragraph:  1
Estimating the Innovator's Dilemma: Structural Analysis of Creative Destruction in the Hard Disk Drive Industry, 1981-1998.
number_of_keywords_in_paragraph:  1
Estimating the Innovator's Dilemma: Structural Analysis of Creative Destruction in the Hard Disk Drive Industry, 1981-1998.
number_of_keywords_in_paragraph:  1
Emotions and Political Unrest.
number_of_keywords_in_paragraph:  1
Emotions and Political Unrest.
number_of_keywords_in_paragraph:  1
Curriculum and Ideology.
number_of_keywords_in_paragraph:  1
Curriculum and Ideology.
number_of_keywords_in_paragraph:  1
Economic Uncertainty, Parental Selection, and Children's Educational

Regional Heterogeneity and the Refinancing Channel of Monetary Policy *.
number_of_keywords_in_paragraph:  3
Regional Heterogeneity and the Refinancing Channel of Monetary Policy *.
number_of_keywords_in_paragraph:  2
Regional Heterogeneity and the Refinancing Channel of Monetary Policy *.
number_of_keywords_in_paragraph:  1
Regional Heterogeneity and the Refinancing Channel of Monetary Policy *.
number_of_keywords_in_paragraph:  1
Regional Heterogeneity and the Refinancing Channel of Monetary Policy *.
number_of_keywords_in_paragraph:  1
Regional Heterogeneity and the Refinancing Channel of Monetary Policy *.
number_of_keywords_in_paragraph:  1
Regional Heterogeneity and the Refinancing Channel of Monetary Policy *.
number_of_keywords_in_paragraph:  1
Regional Heterogeneity and the Refinancing Channel of Monetary Policy *.
number_of_keywords_in_paragraph:  3
Regional Heterogeneity and the Refinancing Channel of Monetary Policy *.
number_of_keywords_in_paragraph:  1
Regional Heterogene

number_of_keywords_in_paragraph:  1
The Aggregate Implications of Mergers and Acquisitions.
number_of_keywords_in_paragraph:  1
The Aggregate Implications of Mergers and Acquisitions.
number_of_keywords_in_paragraph:  1
The Aggregate Implications of Mergers and Acquisitions.
number_of_keywords_in_paragraph:  2
Germs, Social Networks, and Growth.
number_of_keywords_in_paragraph:  2
Germs, Social Networks, and Growth.
number_of_keywords_in_paragraph:  1
Structural Change in Investment and Consumption—A Unified Analysis.
number_of_keywords_in_paragraph:  1
Structural Change in Investment and Consumption—A Unified Analysis.
number_of_keywords_in_paragraph:  1
Structural Change in Investment and Consumption—A Unified Analysis.
number_of_keywords_in_paragraph:  2
Monetary Policy for a Bubbly World.
number_of_keywords_in_paragraph:  1
Trade and Domestic Production Networks.
number_of_keywords_in_paragraph:  1
Trade and Domestic Production Networks.
number_of_keywords_in_paragraph:  1
Housing 

Optimal Fiscal Policy with Recursive Preferences.
number_of_keywords_in_paragraph:  4
Optimal Fiscal Policy with Recursive Preferences.
number_of_keywords_in_paragraph:  2
Optimal Fiscal Policy with Recursive Preferences.
number_of_keywords_in_paragraph:  1
Better Lucky Than Rich? Welfare Analysis of Automobile Licence Allocations in Beijing and Shanghai.
number_of_keywords_in_paragraph:  1
The Role of Firm Factors in Demand, Cost, and Export Market Selection for Chinese Footwear Producers.
number_of_keywords_in_paragraph:  1
Attention Variation and Welfare: Theory and Evidence from a Tax Salience Experiment.
number_of_keywords_in_paragraph:  2
Attention Variation and Welfare: Theory and Evidence from a Tax Salience Experiment.
number_of_keywords_in_paragraph:  1
Attention Variation and Welfare: Theory and Evidence from a Tax Salience Experiment.
number_of_keywords_in_paragraph:  1
Attention Variation and Welfare: Theory and Evidence from a Tax Salience Experiment.
number_of_keywords_i

## Tests

In [19]:
paragraph_chunks_formatted = get_formatted_paragraph_chunks(keywordlist, "channel acts in the same direction as the substitution channel, and with comparable magnitude provided that ​σ​ is between ​0.1​and ​0.4​. Hence, representative-agent anal-yses that abstract from redistribution may fail to capture an important reason why real interest rates affect consumption, especially if ​σ​ is small.3 Similarly, across datasets, the covariance between MPCs and NNPs is negative on average. This implies that consumption tends to increase with inflation as a result of the Fisher channel. However, when cast in terms of elasticities, the magnitude is small: an unexpected 1 percent permanent increase in the price level raises con-sumption today by no more than 0.1 percent. This suggests that, while changes in monetary policy can entail significant nominal redistribution, the aggregate effect of this redistribution on consumption is likely to be modest. Finally, in line with previous literature, I estimate the covariance between MPCs and incomes to be negative in the data. If, in addition, low-income agents dispropor-tionately benefit from increases in aggregate income, as suggested, for example, by Coibion et al. (2017), the earnings heterogeneity channel also amplifies the effects of monetary policy. Future work can build on these empirical results in two ways: by providing more precise measures of exposures across groups of agents or regions to inform the debate on the winners and losers from changes in monetary policy, and by estimat-ing the sufficient statistics more precisely in administrative data to help quantify the aggregate effect of this redistribution.4 A rapidly growing literature analyzes the effects of monetary policy in dynamic stochastic general equilibrium models with rich heterogeneity, matching vari-ous aspects of the cross section such as the wealth distribution. Prominent exam-ples include Gornemann, Kuester, and Nakajima (2016); McKay, Nakamura, and Steinsson (2016); and Kaplan, Moll, and Violante (2018). These structural mod-els overcome a number of important limitations of my sufficient statistics approach. They can study the role of investment, analyze the precise interaction between mon-etary and fiscal policy, and explore the effect of shocks that are persistent and/or announced in advance. My paper makes two contributions to this literature. First, I introduce a decomposition of the monetary policy transmission mechanism into its various sources of effects on consumption that is useful to shed light on the under-lying mechanisms in any such model (see Kaplan, Moll, and Violante 2018 for an influential application). Second, I argue that sufficient statistics can discipline the construction of these models. By making sure that the model’s sufficient statistics match the data, researchers can ensure that, even if the model is misspecified, its predictions for the response of consumption to shocks are consistent with the empir-ical evidence. This paper is motivated by an extensive empirical literature documenting that MPCs are large and heterogeneous in the population (see Jappelli and Pistaferri 2010", True)

number_of_keywords_in_paragraph:  4
paragraph_chunks:  ['', 'channel', ' acts in the same direction as the substitution ', 'channel', ', and with comparable magnitude provided that \u200bσ\u200b is between \u200b0.1\u200band \u200b0.4\u200b. Hence, representative-agent anal-yses that abstract from redistribution may fail to capture an important reason why real interest rates affect consumption, especially if \u200bσ\u200b is small.3 Similarly, across datasets, the covariance between MPCs and NNPs is negative on average. This implies that consumption tends to increase with inflation as a result of the Fisher ', 'channel', '. However, when cast in terms of elasticities, the magnitude is small: an unexpected 1 percent permanent increase in the price level raises con-sumption today by no more than 0.1 percent. This suggests that, while changes in monetary policy can entail significant nominal redistribution, the aggregate effect of this redistribution on consumption is likely to be modest.

In [4]:
# Initial function to split paragraphs by 1 keyword, simplistic
def split_paragraph_by_keyword_simple(keyword, paragraph):
    paragraph = re.split(keyword, paragraph, flags=re.IGNORECASE)
    return paragraph

In [293]:
split_paragraph_by_keywordlist(keywordlist, "ausdhijkchannelasd chan channels import-CHANNEL-s channe-l channel-effect")

number_of_keywords_in_paragraph:  4


['ausdhijk',
 'channel',
 'asd chan ',
 'channel',
 's import-',
 'CHANNEL',
 '-s channe-l ',
 'channel',
 '-effect']

In [255]:
any('channela'.lower() == keyword for keyword in keywordlist)

False

In [256]:
any('channel'.lower() == keyword for keyword in keywordlist)

True

In [254]:
any('CHANNEL'.lower() == keyword for keyword in keywordlist)

True

In [80]:
#Test
split_paragraph_by_keyword(keyword, "ausdhijkchannelasd chan channels import-CHANNEL-s channe-l channel-effect")

['ausdhijk', 'asd chan ', 's import-', '-s channe-l ', '-effect']

In [104]:
# Test
get_paragraph_segments(keyword, "ausdhijkchannelasd chan channels import-CHANNEL-s channe-l channel-effect")

['ausdhijk', 'asd chan ', 's import-', '-s channe-l ', '-effect']
[<xlsxwriter.format.Format object at 0x00000166387CBAC0>, 'ausdhijk', <xlsxwriter.format.Format object at 0x0000016638C12DC0>, 'channel', <xlsxwriter.format.Format object at 0x00000166387CBAC0>, 'asd chan ', <xlsxwriter.format.Format object at 0x0000016638C12DC0>, 'channel', <xlsxwriter.format.Format object at 0x00000166387CBAC0>, 's import-', <xlsxwriter.format.Format object at 0x0000016638C12DC0>, 'channel', <xlsxwriter.format.Format object at 0x00000166387CBAC0>, '-s channe-l ', <xlsxwriter.format.Format object at 0x0000016638C12DC0>, 'channel', <xlsxwriter.format.Format object at 0x00000166387CBAC0>, '-effect']


 'ausdhijk',
 'channel',
 'asd chan ',
 'channel',
 's import-',
 'channel',
 '-s channe-l ',
 'channel',
 '-effect']

In [105]:
get_paragraph_segments(keyword, "aaachannel")

['aaa', '']
[<xlsxwriter.format.Format object at 0x00000166387CBAC0>, 'aaa', <xlsxwriter.format.Format object at 0x0000016638C12DC0>, 'channel', <xlsxwriter.format.Format object at 0x00000166387CBAC0>]


 'aaa',
 'channel',

In [118]:
paragraph0 = channels_df.iloc[0, :]['Paragraph']
paragraph0

'orthogonal to productivity, even though the only source of exogenous variation are sector-specific productivity shocks. A recent literature has demonstrated that, under certain conditions, production networks can lead firm-or sector-specific shocks to generate aggregate fluctua-tions, e.g., Horvath (2000), Carvalho (2010), Acemoglu et al. (2012), Carvalho and Gabaix (2013), Baqaee and Fahri (2019), and Carvalho and Grassi (2019). Shocks to a single sector or firm propagate to other sectors or firms through the trade of intermediate inputs. Foerster, Sartre, and Watson (2011) and Atalay (2017) quan-tify these channels, and their results suggest that sector-specific shocks can explain a substantial portion of observed aggregate output fluctuations. However, trade in intermediate inputs by itself does not induce enough correlation in production across sectors to account for all of the observed volatility of aggregate output. We show that news media can serve as a powerful additional sour

In [20]:
paragraph0_split = split_paragraph_by_keyword(keyword, paragraph0)
paragraph0_split

['orthogonal to productivity, even though the only source of exogenous variation are sector-specific productivity shocks. a recent literature has demonstrated that, under certain conditions, production networks can lead firm-or sector-specific shocks to generate aggregate fluctua-tions, e.g., horvath (2000), carvalho (2010), acemoglu et al. (2012), carvalho and gabaix (2013), baqaee and fahri (2019), and carvalho and grassi (2019). shocks to a single sector or firm propagate to other sectors or firms through the trade of intermediate inputs. foerster, sartre, and watson (2011) and atalay (2017) quan-tify these ',
 's, and their results suggest that sector-specific shocks can explain a substantial portion of observed aggregate output fluctuations. however, trade in intermediate inputs by itself does not induce enough correlation in production across sectors to account for all of the observed volatility of aggregate output. we show that news media can serve as a powerful additional sourc

2

In [24]:
paragraph_segments = []
for index, chunk in enumerate(paragraph0_split):
    paragraph_segments.append(chunk)
    if index < len(paragraph0_split) - 1:
        paragraph_segments.append(bold_format_paragraph)
        paragraph_segments.append(keyword)    
print(paragraph_segments)

['orthogonal to productivity, even though the only source of exogenous variation are sector-specific productivity shocks. a recent literature has demonstrated that, under certain conditions, production networks can lead firm-or sector-specific shocks to generate aggregate fluctua-tions, e.g., horvath (2000), carvalho (2010), acemoglu et al. (2012), carvalho and gabaix (2013), baqaee and fahri (2019), and carvalho and grassi (2019). shocks to a single sector or firm propagate to other sectors or firms through the trade of intermediate inputs. foerster, sartre, and watson (2011) and atalay (2017) quan-tify these ', <xlsxwriter.format.Format object at 0x00000166375556D0>, 'channel', 's, and their results suggest that sector-specific shocks can explain a substantial portion of observed aggregate output fluctuations. however, trade in intermediate inputs by itself does not induce enough correlation in production across sectors to account for all of the observed volatility of aggregate outpu

C:\Users\jasonjia\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:338: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
